In [ ]:
dir_drive = "/content/drive"

params = "(210,2,6,240,320)[180,255,255]"

dir_data_train = f"/content/drive/MyDrive/dataset_signature/train_image{params}.pkl"
dir_data_test = "/content/drive/MyDrive/dataset_signature/test_image(150,2,4,240,320)[180,255,255].pkl"

dir_label = "/content/drive/MyDrive/dataset_signature/label_signature.pkl"

dir_model_save = ["/content/drive/MyDrive/dataset_signature/model_signature[",f"]{params}.keras"]
dir_epochs = ["/content/drive/MyDrive/dataset_signature/model_signature_epochs[",f"]{params}.pkl"]

In [ ]:
model_epochs = 0

history=None

In [ ]:
from google.colab import drive

# Mount Google Drive
drive.mount(dir_drive)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pickle


def import_pkl(file_path):
    # Specify the path to the pickle file

    # Load the pickle file
    try:
        with open(file_path, 'rb') as f:
            loaded_data = pickle.load(f)

        print("Pickle file loaded successfully.")
        # You can now work with 'loaded_data'

        return loaded_data

    except FileNotFoundError:
        print(f"Error: File not found at {file_path}")
    except Exception as e:
        print(f"An error occurred: {e}")

def export_pkl(file_path, data):
    # Specify the path to the pickle file

    # Save the data to the pickle file
    try:
        with open(file_path, 'wb') as f:
            pickle.dump(data, f)
    except Exception as e:
        print(f"An error occurred: {e}")

In [ ]:
import numpy as np


def maik_pairs(data_signature:list[list, list]):
    pairs = []
    labels = []

    def preprocess(img):
        return np.expand_dims(img, axis=-1)

    def append(img1, img2, value):
        pairs.append([img1, img2])
        labels.append(value)

    total_samples = len(data_signature)

    for i in range(total_samples):

        print(f"\rMuestra: [{i+1}/{total_samples}]", end="")

        forge, genuine = data_signature[i]

        for j in range(len(forge)):
            forge[j] = preprocess(forge[j])

        for j in range(len(genuine)):
            genuine[j] = preprocess(genuine[j])

        # 1. Comparaciones genuino vs genuino (misma clase → label 0)
        for i in range(len(genuine)):
            for j in range(i + 1, len(genuine)):
                append(genuine[i], genuine[j], "0")

        # 2. Comparaciones genuino vs forge (clases distintas → label 1)
        for img_genuine in genuine:
            for img_forge in forge:
                append(img_genuine, img_forge, "1")

    print()
    return np.array(pairs), np.array(labels)

In [ ]:
import matplotlib.pyplot as plt

def model_fit(model, train_dataset, test_dataset, verbose=False, fit_epochs=120):
    h = model.fit(
        train_dataset,
        validation_data=test_dataset,
        epochs=fit_epochs,
        verbose=verbose
    )

    model_epochs += fit_epochs

    if history is None:
        history = h
    else:
        history.history['loss'].extend(h.history['loss'])
        history.history['val_loss'].extend(h.history['val_loss'])
        history.history['accuracy'].extend(h.history['accuracy'])
        history.history['val_accuracy'].extend(h.history['val_accuracy'])

    plt.figure()
    plt.plot(history.history['loss'], label='Entrenamiento')
    plt.plot(history.history['val_loss'], label='Validación')
    plt.xlabel('Época')
    plt.ylabel('Pérdida')
    plt.title('Curva de pérdida')
    plt.legend()
    plt.grid(True)
    plt.show()

    export_pkl(dir_epochs[0]+model_epochs+dir_epochs[1], history)

In [ ]:
train_pairs, train_labels = maik_pairs(
    import_pkl(dir_data_train)
)

Pickle file loaded successfully.
Muestra: [210/210]


In [ ]:
test_pairs, test_labels = maik_pairs(
    import_pkl(dir_data_test)
)

Pickle file loaded successfully.
Muestra: [150/150]


# Neural Network

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)

1 Physical GPUs, 1 Logical GPUs


In [ ]:
# from tensorflow.keras.mixed_precision import set_global_policy
# set_global_policy('mixed_float16')

In [ ]:
# Codificar las etiquetas
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)

# Convertir a one-hot (para softmax)
num_classes = len(label_encoder.classes_)
train_labels_cat = to_categorical(train_labels_encoded, num_classes)
test_labels_cat = to_categorical(test_labels_encoded, num_classes)

In [ ]:
export_pkl(dir_label, label_encoder)

In [ ]:
BATCH_SIZE = 8

train_dataset = tf.data.Dataset.from_tensor_slices(((train_pairs[:, 0], train_pairs[:, 1]), train_labels_cat))
train_dataset = train_dataset.shuffle(1024).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

test_dataset = tf.data.Dataset.from_tensor_slices(((test_pairs[:, 0], test_pairs[:, 1]), test_labels_cat))
test_dataset = test_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [ ]:
from keras.saving import register_keras_serializable

@register_keras_serializable()
class L1Distance(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)

    def call(self, inputs):
        x, y = inputs
        return tf.math.abs(x - y)

def build_base_network(input_shape):
    inputs = tf.keras.Input(shape=input_shape)
    x = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', padding='same')(inputs)
    x = tf.keras.layers.MaxPooling2D()(x)

    x = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same')(x)
    x = tf.keras.layers.MaxPooling2D()(x)

    x = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)  # Sustituye Flatten

    x = tf.keras.layers.Dense(128, activation='relu')(x)
    return tf.keras.Model(inputs, x)

def build_siamese_network(input_shape, num_classes):
    base_network = build_base_network(input_shape)

    input_a = tf.keras.Input(shape=input_shape)
    input_b = tf.keras.Input(shape=input_shape)

    processed_a = base_network(input_a)
    processed_b = base_network(input_b)

    l1_distance = L1Distance()([processed_a, processed_b])

    x = tf.keras.layers.Dense(64, activation='relu')(l1_distance)
    x = tf.keras.layers.Dense(32, activation='relu')(x)
    outputs = tf.keras.layers.Dense(num_classes, activation='softmax')(x)

    model = tf.keras.Model(inputs=[input_a, input_b], outputs=outputs)
    return model

In [ ]:
model = build_siamese_network(input_shape=(240, 320, 1), num_classes=num_classes)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
fit_epochs=120
h = model.fit(
    train_dataset,
    validation_data=test_dataset,
    epochs=fit_epochs,
    verbose=True
)

if history is None:
    history = h
else:
    history.history['loss'].extend(h.history['loss'])
    history.history['val_loss'].extend(h.history['val_loss'])
    history.history['accuracy'].extend(h.history['accuracy'])
    history.history['val_accuracy'].extend(h.history['val_accuracy'])

export_pkl(dir_epochs[0]+model_epochs+dir_epochs[1], history)

model_epochs += fit_epochs

Epoch 1/120
1339/1339 ━━━━━━━━━━━━━━━━━━━━ 23s 17ms/step - accuracy: 0.9798 - loss: 0.0570 - val_accuracy: 0.7124 - val_loss: 1.0824
Epoch 2/120
1339/1339 ━━━━━━━━━━━━━━━━━━━━ 24s 18ms/step - accuracy: 0.9759 - loss: 0.0689 - val_accuracy: 0.7170 - val_loss: 1.2927
Epoch 3/120
1339/1339 ━━━━━━━━━━━━━━━━━━━━ 41s 18ms/step - accuracy: 0.9832 - loss: 0.0510 - val_accuracy: 0.7373 - val_loss: 1.3986
Epoch 4/120
1339/1339 ━━━━━━━━━━━━━━━━━━━━ 24s 18ms/step - accuracy: 0.9824 - loss: 0.0440 - val_accuracy: 0.7342 - val_loss: 1.2733
Epoch 5/120
1339/1339 ━━━━━━━━━━━━━━━━━━━━ 23s 18ms/step - accuracy: 0.9855 - loss: 0.0414 - val_accuracy: 0.7261 - val_loss: 1.2207
Epoch 6/120
1339/1339 ━━━━━━━━━━━━━━━━━━━━ 41s 17ms/step - accuracy: 0.9874 - loss: 0.0337 - val_accuracy: 0.7236 - val_loss: 1.2304
Epoch 7/120
1339/1339 ━━━━━━━━━━━━━━━━━━━━ 24s 18ms/step - accuracy: 0.9878 - loss: 0.0343 - val_accuracy: 0.7388 - val_loss: 1.2413
Epoch 8/120
1339/1339 ━━━━━━━━━━━━━━━━━━━━ 24s 18ms/step - accuracy: 

UnboundLocalError: cannot access local variable 'model_epochs' where it is not associated with a value

In [ ]:
import matplotlib.pyplot as plt

plt.figure()
plt.plot(history.history['loss'], label='Entrenamiento')
plt.plot(history.history['val_loss'], label='Validación')
plt.xlabel('Época')
plt.ylabel('Pérdida')
plt.title('Curva de pérdida')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
model.load_weights("/content/drive/MyDrive/dataset_signature/model_signature[240](210,2,6,240,320)[180,255,255]].keras")

In [ ]:
# Split the test_pairs into two separate arrays for each image in the pair
test_pairs_a = test_pairs[:, 0]
test_pairs_b = test_pairs[:, 1]

predictions = model.predict([test_pairs_a, test_pairs_b])
y_pred = np.argmax(predictions, axis=1)
y_true = np.argmax(test_labels_cat, axis=1)  # Use test_labels_cat for evaluation

print("\nReporte de clasificación:")
print(classification_report(y_true, y_pred, target_names=label_encoder.classes_))

print("Matriz de confusión:")
print(confusion_matrix(y_true, y_pred))

104/104 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step

Reporte de clasificación:
              precision    recall  f1-score   support

           0       0.47      0.53      0.50       900
           1       0.81      0.78      0.80      2400

    accuracy                           0.71      3300
   macro avg       0.64      0.65      0.65      3300
weighted avg       0.72      0.71      0.71      3300

Matriz de confusión:
[[ 473  427]
 [ 530 1870]]


# Guardado del modelo y label

In [ ]:
model_epochs = 240

In [ ]:
model.save(dir_model_save[0]+str(model_epochs)+dir_model_save[1])